In [2]:
# importing libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [40]:
# Read in the events data
Indicators = pd.read_csv('../Database/DataSources/Quarterly Public Sector Debt/QPSD.csv')
#Indicators.head()

In [41]:
Indicators.shape

(59789, 16)

In [42]:
Indicators['Country Name'].unique()

array(['Albania', 'Argentina', 'Armenia', 'Australia', 'Austria',
       'Bangladesh', 'Barbados', 'Belgium', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria',
       'Burkina Faso', 'Cambodia', 'Canada', 'Chile', 'China', 'Colombia',
       'Congo, Dem. Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic',
       'El Salvador', 'Estonia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Greece', 'Guatemala', 'Honduras', 'Hungary', 'Iceland',
       'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan',
       'Kenya', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Latvia',
       'Lebanon', 'Lesotho', 'Lithuania', 'Luxembourg', 'Madagascar',
       'Malawi', 'Malaysia', 'Maldives', 'Malta', 'Mauritius', 'Mexico',
       'Moldova', 'Mongolia', 'Morocco', 'Nepal', 'Netherlands',
       'New Zealand', 'Nicaragua', 'Nigeria', 'North Macedonia', 'Norway',
       'Pakistan', 'Papua New Gui

In [43]:
Indicators.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2021Q1 [YR2021Q1]', '2021Q2 [YR2021Q2]', '2021Q3 [YR2021Q3]',
       '2021Q4 [YR2021Q4]', '2022Q1 [YR2022Q1]', '2022Q2 [YR2022Q2]',
       '2022Q3 [YR2022Q3]', '2022Q4 [YR2022Q4]', '2023Q1 [YR2023Q1]',
       '2023Q2 [YR2023Q2]', '2023Q3 [YR2023Q3]', '2023Q4 [YR2023Q4]'],
      dtype='object')

In [44]:
# Filter rows where 'ColumnName' contains any of the keywords
keywords = ["Gross PSD, Budgetary Central Gov., All maturities, All instruments, Nominal Value",
             "Gross PSD, Budgetary Central Gov., Long-term, All instruments, Nominal Value", 
             "Gross PSD, Budgetary Central Gov., Short-term, All instruments, Nominal Value",
             "Gross PSD, Budgetary Central Gov., All maturities, Debt securities + loans, Nominal Value",
             "Gross PSD, Budgetary Central Gov., D1, D2, D3, and D4 categories",
             "Gross PSD, General Gov., All maturities, All instruments, Nominal Value",
             "Gross PSD, General Gov., Long-term and Short-term, All instruments, Nominal Value",
             "Gross PSD, Central Gov., All maturities, All instruments, Nominal Value",
             "Gross PSD, Financial Public Corp., All maturities, All instruments, Nominal Value",
             "Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, Nominal Value",
             "Gross PSD, Public Sector, All maturities, All instruments, Nominal Value",
             "Gross PSD, Total, All maturities, All instruments, Nominal Value",
             "Gross PSD, Total, Long-term and Short-term, All instruments, Nominal Value",
             "Gross PSD, Central Gov., D1, D2, D3, and D4 categories", "Gross PSD, Budgetary Central Gov., All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Budgetary Central Gov., Long-term and Short-term, All instruments, Nominal Value, % of GDP", 
             "Gross PSD, Budgetary Central Gov., All maturities, Debt securities + loans, Nominal Value, % of GDP",
             "Gross PSD, General Gov., All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, General Gov., Long-term and Short-term, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Central Gov., All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Financial Public Corp., All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Public Sector, All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Total, All maturities, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Total, Long-term and Short-term, All instruments, Nominal Value, % of GDP",
             "Gross PSD, Central Gov., D1, D2, D3, and D4 categories, Nominal Value, % of GDP","Gross PSD, Budgetary Central Gov., All maturities, Debt securities, Market value, % of GDP",
             "Gross PSD, General Gov., All maturities, Debt securities, Market value, % of GDP", 
             "Gross PSD, Central Gov., All maturities, Debt securities, Market value, % of GDP",
             "Gross PSD, Financial Public Corp., All maturities, Debt Securities, Market value, % of GDP",
             "Gross PSD, Nonfinancial Public Corp., All maturities, Debt Securities, Market value, % of GDP",
             "Gross PSD, Public Sector, All maturities, Debt Securities, Market value, % of GDP",
             "Gross PSD, Total, All maturities, Debt Securities, Market value, % of GDP","Gross PSD, Budgetary Central Gov., All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, Budgetary Central Gov., All maturities, All instruments, Foreign currency, Nominal Value", 
             "Gross PSD, General Gov., All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, General Gov., All maturities, All instruments, Foreign currency, Nominal Value",
             "Gross PSD, Central Gov., All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, Central Gov., All maturities, All instruments, Foreign currency, Nominal Value",
             "Gross PSD, Financial Public Corp., All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, Financial Public Corp., All maturities, All instruments, Foreign currency, Nominal Value",
             "Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, Foreign currency, Nominal Value",
             "Gross PSD, Public Sector, All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, Public Sector, All maturities, All instruments, Foreign currency, Nominal Value",
             "Gross PSD, Total, All maturities, All instruments, External creditors, Nominal Value",
             "Gross PSD, Total, All maturities, All instruments, Foreign currency, Nominal Value"]  # Replace with actual keywords
public_Debt_Indicators_df  = Indicators[Indicators['Series Name'].str.contains('|'.join(keywords), case=False, na=False)]


In [45]:
#TotalGovernmentDebt_df.head(2)
public_Debt_Indicators_df['Country Name'].unique()

array(['Albania', 'Argentina', 'Armenia', 'Australia', 'Austria',
       'Bangladesh', 'Barbados', 'Belgium', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria',
       'Burkina Faso', 'Cambodia', 'Canada', 'Chile', 'China', 'Colombia',
       'Congo, Dem. Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic',
       'El Salvador', 'Estonia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Greece', 'Guatemala', 'Honduras', 'Hungary', 'Iceland',
       'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan',
       'Kenya', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Latvia',
       'Lebanon', 'Lesotho', 'Lithuania', 'Luxembourg', 'Madagascar',
       'Malawi', 'Malaysia', 'Maldives', 'Malta', 'Mauritius', 'Mexico',
       'Moldova', 'Mongolia', 'Morocco', 'Nepal', 'Netherlands',
       'New Zealand', 'Nicaragua', 'Nigeria', 'North Macedonia', 'Norway',
       'Pakistan', 'Papua New Gui

In [46]:
public_Debt_Indicators_df.shape

(6572, 16)

In [47]:
public_Debt_Indicators_df.isnull().sum()

Country Name         0
Country Code         0
Series Name          0
Series Code          0
2021Q1 [YR2021Q1]    0
2021Q2 [YR2021Q2]    0
2021Q3 [YR2021Q3]    0
2021Q4 [YR2021Q4]    0
2022Q1 [YR2022Q1]    0
2022Q2 [YR2022Q2]    0
2022Q3 [YR2022Q3]    0
2022Q4 [YR2022Q4]    0
2023Q1 [YR2023Q1]    0
2023Q2 [YR2023Q2]    0
2023Q3 [YR2023Q3]    0
2023Q4 [YR2023Q4]    0
dtype: int64

In [48]:
public_Debt_Indicators_df.dtypes

Country Name         object
Country Code         object
Series Name          object
Series Code          object
2021Q1 [YR2021Q1]    object
2021Q2 [YR2021Q2]    object
2021Q3 [YR2021Q3]    object
2021Q4 [YR2021Q4]    object
2022Q1 [YR2022Q1]    object
2022Q2 [YR2022Q2]    object
2022Q3 [YR2022Q3]    object
2022Q4 [YR2022Q4]    object
2023Q1 [YR2023Q1]    object
2023Q2 [YR2023Q2]    object
2023Q3 [YR2023Q3]    object
2023Q4 [YR2023Q4]    object
dtype: object

In [ ]:
# Save result
public_Debt_Indicators_df.to_csv("Public_Debt_Indicators.csv", index=False)

In [50]:
df=pd.read_csv('../WEEK 3/Indicators/Public_Debt_Indicators.csv')

In [51]:
df.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2021Q1 [YR2021Q1]', '2021Q2 [YR2021Q2]', '2021Q3 [YR2021Q3]',
       '2021Q4 [YR2021Q4]', '2022Q1 [YR2022Q1]', '2022Q2 [YR2022Q2]',
       '2022Q3 [YR2022Q3]', '2022Q4 [YR2022Q4]', '2023Q1 [YR2023Q1]',
       '2023Q2 [YR2023Q2]', '2023Q3 [YR2023Q3]', '2023Q4 [YR2023Q4]'],
      dtype='object')

In [ ]:
# converting series name into column 

# Ensure the format keeps Country_Name, Country_Code, and Time as columns
df_pivot = df.pivot(index=['Country Name', 'Country Code', '2021Q1 [YR2021Q1]', '2021Q2 [YR2021Q2]', '2021Q3 [YR2021Q3]',
       '2021Q4 [YR2021Q4]', '2022Q1 [YR2022Q1]', '2022Q2 [YR2022Q2]',
       '2022Q3 [YR2022Q3]', '2022Q4 [YR2022Q4]', '2023Q1 [YR2023Q1]',
       '2023Q2 [YR2023Q2]', '2023Q3 [YR2023Q3]', '2023Q4 [YR2023Q4]'], 
                    columns='Series Name', values='Value').reset_index()




In [58]:

# Melt: Convert quarter-based values into a single column
df_melted = df.melt(id_vars=["Country Name", "Country Code", "Series Name", "Series Code"], 
                     var_name="Time", value_name="Value")



In [59]:

# Extract proper year/quarter format
df_melted["Time"] = df_melted["Time"].str.extract(r"(\d{4}Q\d)")


In [ ]:

# Pivot: Convert 'Series Name' into columns and assign values correctly
df_pivot = df_melted.pivot(index=["Country Name", "Country Code", "Time"], 
                           columns="Series Name", values="Value").reset_index()




In [61]:
df_pivot.head(5)

Series Name,Country Name,Country Code,Time,"Gross PSD, Budgetary Central Gov., All maturities, All instruments, External creditors, Nominal Value, % of GDP","Gross PSD, Budgetary Central Gov., All maturities, All instruments, External creditors, Nominal Value, National Currency","Gross PSD, Budgetary Central Gov., All maturities, All instruments, External creditors, Nominal Value, US$","Gross PSD, Budgetary Central Gov., All maturities, All instruments, Foreign currency, Nominal Value, % of GDP","Gross PSD, Budgetary Central Gov., All maturities, All instruments, Foreign currency, Nominal Value, National Currency","Gross PSD, Budgetary Central Gov., All maturities, All instruments, Foreign currency, Nominal Value, US$","Gross PSD, Budgetary Central Gov., All maturities, All instruments, Nominal Value, % of GDP",...,"Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, Nominal Value, % of GDP","Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, Nominal Value, National Currency","Gross PSD, Nonfinancial Public Corp., All maturities, All instruments, Nominal Value, US$","Gross PSD, Nonfinancial Public Corp., All maturities, Debt Securities, Market value, % of GDP","Gross PSD, Public Sector, All maturities, All instruments, External creditors, Nominal Value, National Currency","Gross PSD, Public Sector, All maturities, All instruments, Foreign currency, Nominal Value, National Currency","Gross PSD, Public Sector, All maturities, All instruments, Nominal Value, National Currency","Gross PSD, Total, All maturities, All instruments, External creditors, Nominal Value, US$","Gross PSD, Total, All maturities, All instruments, Foreign currency, Nominal Value, US$","Gross PSD, Total, All maturities, All instruments, Nominal Value, US$"
0,Albania,ALB,2021Q1,..,..,..,..,..,..,..,...,..,..,..,..,531257123584.814,722793174141.008,1703280043413.86,5063932166.47425,6889649929.85424,16235630954.2833
1,Albania,ALB,2021Q2,..,..,..,..,..,..,..,...,..,..,..,..,519151667435.039,692971928813.674,1690269527643.36,5035418694.81124,6721357214.48762,16394466805.4642
2,Albania,ALB,2021Q3,..,..,..,..,..,..,..,...,..,..,..,..,530065825513.531,718310141143.752,1733367872581.04,5058362682.637,6854758480.2343,16541348149.4517
3,Albania,ALB,2021Q4,..,..,..,..,..,..,..,...,..,..,..,..,611719403332.739,888955963471.537,1911645190276.31,5741687660.34108,8343870503.76888,17942980948.717
4,Albania,ALB,2022Q1,..,..,..,..,..,..,..,...,..,..,..,..,610534312675.763,893394172679.381,1912366924689.04,5586880606.4766,8175276104.31352,17499697334.2701


In [62]:
df_pivot.shape

(1272, 65)

In [66]:
df_pivot['Time'].nunique()

12

In [67]:
# Save the formatted output
df_pivot.to_csv("Public Debt Indicators.csv", index=False)

In [ ]:

# Extract year from the 'Time' column
df_pivot["Year"] = df_pivot["Time"].str.extract(r"(\d{4})")

# Drop the original 'Time' column
df_pivot = df_pivot.drop(columns=["Time"])

# Group by 'Country Name', 'Country Code', and 'Year' to aggregate quarterly data
QPSD = df_pivot.groupby(["Country Name", "Country Code", "Year"], as_index=False).sum()

# Save the final output
QPSD.to_csv("Public_Debt_Indicators (QPSD).csv", index=False)
